In [20]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

base_path = '/home/lab17/jupyter_home/Data/product_image'


path = os.path.join(base_path, 'Training')
path = os.path.join(path, 'image')
files = os.listdir(base_path)
print(files)

# print([int(f[:5]) for f in files])
# print([f[6:] for f in files])

# train_dic = dict(zip([int(f[:5]) for f in files], [f[6:] for f in files]))
# print(train_dic)

['Validation', 'Training']


In [41]:
sorted(glob(os.path.join(base_path, 'Training/image/**/*.jpg')))

['/home/lab17/jupyter_home/Data/product_image/Training/image/10242_Dole후룻볼슬라이스복숭아198g/10242_00_m_1.jpg',
 '/home/lab17/jupyter_home/Data/product_image/Training/image/10242_Dole후룻볼슬라이스복숭아198g/10242_00_m_11.jpg',
 '/home/lab17/jupyter_home/Data/product_image/Training/image/10242_Dole후룻볼슬라이스복숭아198g/10242_00_m_12.jpg',
 '/home/lab17/jupyter_home/Data/product_image/Training/image/10242_Dole후룻볼슬라이스복숭아198g/10242_00_m_13.jpg',
 '/home/lab17/jupyter_home/Data/product_image/Training/image/10242_Dole후룻볼슬라이스복숭아198g/10242_00_m_14.jpg',
 '/home/lab17/jupyter_home/Data/product_image/Training/image/10242_Dole후룻볼슬라이스복숭아198g/10242_00_m_16.jpg',
 '/home/lab17/jupyter_home/Data/product_image/Training/image/10242_Dole후룻볼슬라이스복숭아198g/10242_00_m_17.jpg',
 '/home/lab17/jupyter_home/Data/product_image/Training/image/10242_Dole후룻볼슬라이스복숭아198g/10242_00_m_18.jpg',
 '/home/lab17/jupyter_home/Data/product_image/Training/image/10242_Dole후룻볼슬라이스복숭아198g/10242_00_m_2.jpg',
 '/home/lab17/jupyter_home/Data/product_image/Tr

In [40]:
base_path = '/home/lab17/jupyter_home/Data/product_image'

train_jpg = sorted(glob(os.path.join(base_path, 'Training/image/**/*.jpg')))
valid_jpg = sorted(glob(os.path.join(base_path, 'Validation/image/**/*.jpg')))

train_label = [t_img[len(base_path+'/Training/image/'):len(base_path+'/Training/image/')+5] for t_img in train_jpg]
valid_label = [v_img[len(base_path+'/Validation/image/'):len(base_path+'/Validation/image/')+5] for v_img in valid_jpg]

print('train_label count', len(set(train_label)))
print('valid_label count', len(set(valid_label)))
print('train_label length' , len(train_label))
print('valid_label length' , len(valid_label))


train_label count 76
valid_label count 76
train_label length 8664
valid_label length 1140


In [ ]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
#         if self.mode == 'train':
#             train_transform = transforms.Compose([
#                     transforms.ToTensor(),
#                     transforms.Normalize(mean = [0.433038, 0.403458, 0.394151],
#                                         std = [0.181572, 0.174035, 0.163234]),
#                     transforms.RandomAffine((-45, 45)),

#                     transforms.RandomVerticalFlip(p=0.5),   # - 이미지를 랜덤으로 수직으로 뒤집는다. p =0이면 뒤집지 않는다.
#                     transforms.RandomHorizontalFlip(p=0.5), # - 이미지를 랜덤으로 수평으로 뒤집는다.
#                     transforms.RandomRotation((0,80))       #  이미지를 랜덤으로 degrees 각도로 회전한다.

#                 ])
#                 img = train_transform(img)
#         if self.mode == 'test':
#           test_transform = transforms.Compose([
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean = [0.418256, 0.393101, 0.386632],
#                                      std = [0.195055, 0.190053, 0.185323]),
              
#             ])
#           img = test_transform(img)

        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self,mode = 'train'):
        super(Network, self).__init__()
        self.mode = mode
        if self.mode == 'train':
          self.model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=88, drop_path_rate = 0.2)
        if self.mode == 'test':
          self.model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=88, drop_path_rate = 0)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
# def score_function(real, pred):
#     score = f1_score(real, pred, average="macro")
#     return score

In [ ]:
def main(seed = 2022):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
    
main(2022)

In [ ]:
# Train
train_dataset = Custom_dataset(train_jpg, train_label, mode='train')
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

# Val
val_dataset = Custom_dataset(valid_jpg, valid_label, mode='test')
val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)


